In [1]:
from utils import (
    RealDescription,
    LoveNumbersHyperParameters,
    load_base_model,
    parameters_path,
    Earth_radius,
    elastic_Love_numbers_computing,
)

Love_numbers_hyper_parameters: LoveNumbersHyperParameters = load_base_model(
    name="Love_numbers_hyper_parameters", path=parameters_path, base_model_type=LoveNumbersHyperParameters
)
Love_numbers_hyper_parameters.load()
real_description = RealDescription(
    id="test-Asthenosphere",
    below_ICB_layers=Love_numbers_hyper_parameters.real_description_parameters.below_ICB_layers,
    below_CMB_layers=Love_numbers_hyper_parameters.real_description_parameters.below_CMB_layers,
    splines_degree=Love_numbers_hyper_parameters.real_description_parameters.splines_degree,
    radius_unit=(
        Love_numbers_hyper_parameters.real_description_parameters.radius_unit
        if Love_numbers_hyper_parameters.real_description_parameters.radius_unit
        else Earth_radius
    ),
    real_crust=Love_numbers_hyper_parameters.real_description_parameters.real_crust,
    n_splines_base=10,
    profile_precision=100,
    radius=(
        Love_numbers_hyper_parameters.real_description_parameters.radius_unit
        if Love_numbers_hyper_parameters.real_description_parameters.radius_unit
        else Earth_radius
    ),
    load_description=False,
    anelasticity_model_from_name="test-low-viscosity-Asthenosphere",
)

In [2]:
from utils import Result

degrees = [1, 2]
elastic_Love_numbers = elastic_Love_numbers_computing(
    y_system_hyper_parameters=Love_numbers_hyper_parameters.y_system_hyper_parameters,
    degrees=degrees,
    real_description=real_description,
)

elastic_result = Result(hyper_parameters=Love_numbers_hyper_parameters)
elastic_result.update_values_from_array(result_array=elastic_Love_numbers, degrees=degrees)
elastic_result.values

{<Direction.radial: 0>: {<BoundaryCondition.load: 0>: array([[-0.28579981+0.j],
         [-0.9911582 +0.j]]),
  <BoundaryCondition.shear: 1>: array([[0.        +0.j],
         [0.06054103+0.j]]),
  <BoundaryCondition.potential: 2>: array([[0.        +0.j],
         [0.60322341+0.j]])},
 <Direction.tangential: 1>: {<BoundaryCondition.load: 0>: array([[0.10364662+0.j],
         [0.04666531+0.j]]),
  <BoundaryCondition.shear: 1>: array([[0.        +0.j],
         [0.63388481+0.j]]),
  <BoundaryCondition.potential: 2>: array([[0.        +0.j],
         [0.16774736+0.j]])},
 <Direction.potential: 2>: {<BoundaryCondition.load: 0>: array([[ 0.        +0.j],
         [-0.61048221+0.j]]),
  <BoundaryCondition.shear: 1>: array([[0.        +0.j],
         [0.16774735+0.j]]),
  <BoundaryCondition.potential: 2>: array([[0.       +0.j],
         [0.5959646+0.j]])}}